# Loading

In [1]:
import autocompleter 
autocompl = autocompleter.Autocompleter()

In [2]:
df = autocompl.import_json("sample_conversations.json")
df.shape, df.columns

load json file...
(22264, 3)


((22264, 3), Index(['IsFromCustomer', 'Text', 'index'], dtype='object'))

The file contains 22K conversations between a customer and a representative.
For the purpose of this project, we are only interested in completing the threads of the representative.

In [4]:
df.head()

,IsFromCustomer,Text,index
0,True,Hi! I placed an order on your website and I ca...,0
1,True,I think I used my email address to log in.,0
2,True,My battery exploded!,1
3,True,"It's on fire, it's melting the carpet!",1
4,True,What should I do!,1


# Data Selection and Cleaning

The data is going to separate the threads from the customer and the representative, separate the sentenses based on the punctuation (we will keep the punctuation), the final text will be cleaned up with some light regex and only the sentense larger than 1 word will be kept.

Finally, since the representative has the tendency to ask the same question over and over again, the autocomplete is extremely useful by suggesting a complete sentense. In our case, we will count the number of occurence of the same sentense so we can use it as a feature later on and delete the duplicates.

In [5]:
new_df = autocompl.process_data(df)
new_df.shape, new_df.columns

select representative threads...
split sentenses on punctuation...
Text Cleaning using simple regex...
calculate nb words of sentenses...
count occurence of sentenses...
remove duplicates (keep last)...
(8599, 5)


((8599, 5),
 Index(['IsFromCustomer', 'Text', 'index', 'nb_words', 'Counts'], dtype='object'))

# Model and TFIDF matrix

A matrice of similarity is calculated based on the frequency of all the words in the data using tfidfvectorizer

In [6]:
model_tf, tfidf_matrice = autocompl.calc_matrice(new_df)

tfidf_matrice  (8599, 99395)


# Ranking Function

Finally, the autocomplete is calculating the similarity between the sentense in the data and the prefix of the sentense written by the representative. As a weight feature, we chose to reorder using the frequency of the most common similar sentense.

examples of auto completions

In [28]:
prefix = 'What is your'

print(prefix,"    \n ")

autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

What is your     
 


['What is your account number?',
 'What is your order number?',
 'What is your phone number?']

In [22]:
prefix = 'How can'
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

How can      


['How can I help you?',
 'How can I help you today?',
 'Ok lets see how I can help']

In [29]:
prefix = 'Let me'
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

Let me      


['Let me investigate', 'Let me assist you', 'Let me look']

In [31]:
prefix = 'when was'
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

when was      


['When was the last time you changed your password?',
 'When was your flight scheduled for?',
 'When was the last time you tried?']

Now, without any uppercase and just with the important words...

In [35]:
prefix = 'when time password'
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

when time password      


['When was the last time you changed your password?',
 'When you select you password?',
 'Take your time']

# Online Sources for this project

In [34]:
# https://gist.github.com/jlln/338b4b0b55bd6984f883 modified to keep punctuation
# kaggle google store competition for json read
# https://www.kaggle.com/hamishdickson/weighted-word-autocomplete-using-star-wars-dataset

In [ ]:
#